In [ ]:
import requests
import pandas as pd
import time

In [ ]:
# 获取所有品牌列表

def get_all_brands():
    r = requests.get('https://www.dongchedi.com/motor/pc/car/brand/all_brand')
    data = r.json()['data']

    brand_list = []
    for brand in data['brand']:
        if brand['type'] == 1000:
            continue
        item = {**brand['info']}
        del item['image_url']
        del item['brand_activity_tag']
        del item['discount_tag']
        brand_list.append(item)
    return brand_list

brand_list = get_all_brands()
print(len(brand_list))

In [ ]:
# 保存一下所有品牌列表
df = pd.DataFrame(brand_list)
df.to_csv('./data/all_brands.csv', index=None)

In [ ]:
# 获取每一个牌子的所有车型

def get_brand_data(brand_id):
    brand_url = 'https://www.dongchedi.com/motor/pc/car/brand/get_brand_series_list'
    params = {
        'brand_id': brand_id,
        'no_sales': 0,
        'show_historical_series': 1,
        'category_group_optimize': 'true'
    }
    response = requests.get(brand_url, params=params)
    try:
        data = response.json()['data']['category_list'][0]
        # 判断一下是不是拿的全部
        if data['category_name'] != '全部':
            print('获取的目录不是全部 -- id:{}'.format(brand_id))
        return data['list']
    except:
        print('请求出错 -- id:{}'.format(brand_id))
        return []

In [ ]:
# 保存一个牌子所有的车到csv

def save_brand_data(brand_id, brand_name):
    # 获取当前牌子的所有车
    brand_data_list = get_brand_data(brand_id)

    car_type = ''   # 记录车的类型
    brand_items = []  # 重新保存一下列表

    for item in brand_data_list:
        # 这个只是用于声明车的类型
        if item['type'] == 1075:
            car_type = item['info']['text']

        # 判断是否存在忽略了的类型
        elif item['type'] != 1002:
            print('Error type is ' + item['type'])

        # 保存该车的信息
        else:
            item_info = item['info']
            item_info['unique_id'] = item['unique_id']
            item_info['brand_id'] = brand_id
            item_info['brand_name'] = brand_name
            item_info['car_type'] = car_type
            brand_items.append(item_info)

    # 保存成csv
    brand_df = pd.DataFrame(brand_items)
    brand_df.to_csv('./data/brand_{}.csv'.format(brand_id), index=None)



In [ ]:
for index, brand in enumerate(brand_list):
    brand_id = brand['brand_id']
    brand_name = brand['brand_name']

    save_brand_data(brand_id, brand_name)
    print('[Info]: 完成 {} -- id:{}'.format(index, brand_id))

    # 等个两秒钟
    time.sleep(2)
